# 量子アニーリングマシンでスーパコンピュータ設計
## 背景
スーパーコンピュータおよびHPC(High Performance Computing)クラスタ（以下、HPCシステム）は、CPU、メモリ、アクセラレータ、インターコネクトなど様々な部品から構成される。
システム設計を行う際、予算および電力容量内で、要求される性能（理論演算性能、コア数、シングルスレッド性能、ノード数、アクセラレータの有無など計算対象の分野によって優先度が異なる）最大化するシステム設計が必要となる。人手で全ての細かい部品について入れ替えて検討することは、組み合わせ数が膨大であり、仕様策定の限られた期間内では困難である。
そこで、このHPCシステム設計を最適化問題として捉え、ヒューリスティックな解法により求める性能の最適構成を高速に手に入れることを目的とする。

この手法は、入力するデータを変更することでHPCシステムに限らず、様々な構成を提案することができる。
D-Waveマシンなどと連携した高速な構成提案システムを検討する。


# CPUに関して検討してみる
様々な部品の組み合わせが考えられるが、計算機ノード内の性能や消費電力に大きく影響を与える主要な部品であるCPUの組み合わせを検討してみる。

## CPUデータの取得
### Intel
Processor関係の製品情報の比較ページでデータをエクスポートすることができる。このデータをtsv形式で保存し、pandasで読み込んで利用する。

https://www.intel.in/content/www/in/en/products/compare.html?productIds=217216,212282,212284,212287,212288,212289,212307,212454,212455,212459,217215,212308,205688,204089,204097,205683,204087,204092,204094,204096,205684,195437,195434,194146,194145,192465,192467,192470,192472,192474,192475,192478,192480,192481,192482,192476,120497,120499,120500,120501,120503,120504,120506,120508,123543,123687,125056,120496


### AMD
プロセッサー仕様のWebページから、csvデータを取得できる。このデータをpandasで読み込んで利用する。

https://www.amd.com/ja/products/specifications/processors

In [1]:
import pandas as pd

In [4]:
# データ読み込み

## intel
df_csv_intel = pd.read_csv('data/Intel_UPE_ComparisonChart_2022_04_13.csv', skiprows=1, header=0, index_col=0)

## AMD
df_csv_amd = pd.read_csv('data/AMD_EPYC_Comparison_2022_04_13.csv', header=0, index_col=0)


In [5]:
df_csv_intel

,Intel® Xeon® Scalable Processors,Intel® Xeon® Scalable Processors.1,Intel® Xeon® Scalable Processors.2,Intel® Xeon® Scalable Processors.3,Intel® Xeon® Scalable Processors.4,Intel® Xeon® Scalable Processors.5,Intel® Xeon® Scalable Processors.6,Intel® Xeon® Scalable Processors.7,Intel® Xeon® Scalable Processors.8,Intel® Xeon® Scalable Processors.9,...,3rd Generation Intel® Xeon® Scalable Processors.11,3rd Generation Intel® Xeon® Scalable Processors.12,3rd Generation Intel® Xeon® Scalable Processors.13,3rd Generation Intel® Xeon® Scalable Processors.14,3rd Generation Intel® Xeon® Scalable Processors.15,3rd Generation Intel® Xeon® Scalable Processors.16,3rd Generation Intel® Xeon® Scalable Processors.17,3rd Generation Intel® Xeon® Scalable Processors.18,3rd Generation Intel® Xeon® Scalable Processors.19,3rd Generation Intel® Xeon® Scalable Processors.20
Product Collection,,,,,,,,,,,,,,,,,,,,,
Vertical Segment,Server,Server,Server,Server,Server,Server,Server,Server,Server,Server,...,Server,Server,Server,Server,Server,Server,Server,Server,Server,Server
Processor Number,8180,8153,8156,8158,8160,8164,8168,8170,8176,8160T,...,8380,8351N,8368Q,8352S,8358P,8352V,8368,8360Y,8352M,8362
Status,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,...,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched,Launched
Launch Date,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,Q3'17,...,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21,Q2'21
Lithography,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,14 nm,...,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm,10 nm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Intel® Software Guard Extensions (Intel® SGX),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS,Yes with Intel® SPS
Intel® Total Memory Encryption,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
Intel® Platform Firmware Resilience Support,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes


In [6]:
df_csv_amd

,Unnamed: 1,モデル,シリーズ,ライン,プラットフォーム,Product ID Tray,Product ID Boxed,Product ID MPK,発売日,CPUコア数,...,*OS サポート,System Memory Specification,System Memory Type,メモリー・チャネル,Per Socket Mem BW,グラフィックス周波数,GPUベース,グラフィックス・モデル,サポートされるテクノロジー,ワークロードの類似性
NaN,on,AMD EPYC™ 7773X,AMD EPYC™,AMD EPYC™ 7003 Series with AMD 3D V-Cache™,サーバー,100-000000504​,100-000000504​WOF,NaN,NaN,64,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,"AMD Infinity Guard, AMD Infinityアーキテクチャー","Cache-sensitive scale-up/out, CFD, FEA Solvers"
NaN,on,AMD EPYC™ 7763,AMD EPYC™,AMD EPYC™ 7003 Series,サーバー,100-000000312,100-100000312WOF,NaN,3/15/2021,64,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,"AMD Infinity Guard, AMD Infinityアーキテクチャー","Analytics, CAE/CFD/FEA, ERM/SCM/CRM apps, High..."
NaN,on,AMD EPYC™ 7742,AMD EPYC™,AMD EPYC™ 7002 Series,サーバー,100-000000053,100-100000053WOF,NaN,NaN,64,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,AMD Infinityアーキテクチャー,"Analytics, CAE/CFD/FEA, ERM/SCM/CRM apps, High..."
NaN,on,AMD EPYC™ 7713P,AMD EPYC™,AMD EPYC™ 7003 Series,サーバー,100-000000337,100-100000337WOF,NaN,3/15/2021,64,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,"AMD Infinity Guard, AMD Infinityアーキテクチャー","App dev/test, ERM/SCM/CRM apps, Value data mgm..."
NaN,on,AMD EPYC™ 7713,AMD EPYC™,AMD EPYC™ 7003 Series,サーバー,100-000000344,100-100000344WOF,NaN,3/15/2021,64,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,"AMD Infinity Guard, AMD Infinityアーキテクチャー","Analytics, App dev/test, CAE/CFD/FEA, ERM/SCM/..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,on,AMD EPYC™ 72F3,AMD EPYC™,AMD EPYC™ 7003 Series,サーバー,100-000000327,100-100000327WOF,NaN,3/15/2021,8,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,"AMD Infinity Guard, AMD Infinityアーキテクチャー","Departmental CAE/CFD/FEA, EDA, FSI risk analys..."
NaN,on,AMD EPYC™ 7H12,AMD EPYC™,AMD EPYC™ 7002 Series,サーバー,100-000000055,100-100000055WOF,NaN,NaN,64,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,AMD Infinityアーキテクチャー,Specialized CAE/CFD/FEA
NaN,on,AMD EPYC™ 7F72,AMD EPYC™,AMD EPYC™ 7002 Series,サーバー,100-000000141,100-000000141WOF,NaN,NaN,24,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,"AMD Infinity Guard, AMD Infinityアーキテクチャー","EDA, HCI, High performance VM Density, Per cor..."
NaN,on,AMD EPYC™ 7F52,AMD EPYC™,AMD EPYC™ 7002 Series,サーバー,100-000000140,100-000000140WOF,NaN,NaN,16,...,NaN,Up to 3200MHz,DDR4,8,204.80 GB/s,NaN,NaN,NaN,"AMD Infinity Guard, AMD Infinityアーキテクチャー","EDA, High-speed data mgmt (NR/RDBMS), Per core..."


# 線形計画法/D-Waveで解くアイディア

## 線形計画法
objective:
- max 性能
- max ノード数

subject to:
- 合計消費電力<最大消費電力
- 合計金額<予算

決定変数:
- CPUの種類
- メモリの容量と枚数
- その他、選択可能な部品